# FRI: Fast Reed Solomon IOP of Proximity

---
## FRI Low Degree Tests
---

In [1]:
%load_ext autoreload
%autoreload 2

from src.proth_primes import proth_in_range
from src.fri import FRIOPP, FRIPCS_Prover, FRIPCS_Verifier
from sage.rings.finite_rings.all import GF
from sage.rings.polynomial.all import PolynomialRing
from sage.misc.functional import log
from sage.misc.prandom import random

In [2]:
(p, k, n) = proth_in_range(15, 15, 27,27)
print(f"Using prime {p} = {k}⋅2^{n} + 1")
Fq = GF(p)
omega = Fq.multiplicative_generator() 
omega = omega**k
omega_2sylow_expo = log(omega.multiplicative_order(), 2)
print(f"Maximal 2-sylow multiplicative generator: {omega}")
print(f"Maximal 2-sylow multiplicative power: {omega_2sylow_expo}")

Using prime 2013265921 = 15⋅2^27 + 1
Maximal 2-sylow multiplicative generator: 440564289
Maximal 2-sylow multiplicative power: 27


### Setup

In [3]:
polynomial_degree = 512
folding_factor = 4
rate_factor = 4
rs_code_length = rate_factor*polynomial_degree
rs_code_log_len = int(log(rs_code_length, 2))
PolyRing = PolynomialRing(Fq, "X")
ntt_omega = omega**(2**(omega_2sylow_expo - rs_code_log_len))
assert ntt_omega.multiplicative_order() == rs_code_length

input_polynomial = PolyRing.random_element(polynomial_degree-1)

print(f"Using polynomial: {input_polynomial}")

Using polynomial: 189446473*X^511 + 1887525604*X^510 + 1995045324*X^509 + 92468193*X^508 + 37250970*X^507 + 1782909639*X^506 + 1097782517*X^505 + 1792228040*X^504 + 1508883015*X^503 + 1604780547*X^502 + 831912909*X^501 + 170663884*X^500 + 208740638*X^499 + 1096769607*X^498 + 1320297757*X^497 + 1210580977*X^496 + 686744765*X^495 + 1912317870*X^494 + 583015882*X^493 + 609794307*X^492 + 1143539288*X^491 + 238884464*X^490 + 414376753*X^489 + 1577859556*X^488 + 1478831601*X^487 + 320044573*X^486 + 1068683403*X^485 + 1643769744*X^484 + 968826857*X^483 + 713524189*X^482 + 1559677840*X^481 + 1808414572*X^480 + 616753196*X^479 + 369442567*X^478 + 1281367581*X^477 + 1674756031*X^476 + 1174814323*X^475 + 1200755310*X^474 + 1788093022*X^473 + 1396156594*X^472 + 952892639*X^471 + 235193249*X^470 + 1027910567*X^469 + 1525610627*X^468 + 353148555*X^467 + 94058086*X^466 + 1030223912*X^465 + 1156090947*X^464 + 191786119*X^463 + 628689551*X^462 + 1183025583*X^461 + 527879846*X^460 + 1544754304*X^459 + 1

### Low degree test

In [4]:
iopp = FRIOPP(input_polynomial=input_polynomial, 
              ntt_omega=ntt_omega, 
              rate_factor=rate_factor, 
              folding_factor=folding_factor)

max_degree = polynomial_degree

if iopp.verify_proximity(max_degree=max_degree) is True:
    print("✓ Positive case of low degree test passed")
else:
    print("❌ERROR: Positive case of low dergee test failed!")

if iopp.verify_proximity(max_degree // folding_factor  ) is False:
    print("✓ Negative case case of low degree test passed")
else:
    print("❌ ERROR: Negative case of low dergee test failed!")


✓ Positive case of low degree test passed
✓ Negative case case of low degree test passed


---
## FRI Polynomial Commitment Scheme
---

In [5]:
evaluation_point = PolyRing.base_ring().random_element()

prover = FRIPCS_Prover(ntt_omega, rate_factor, folding_factor)
commitment = prover.commit(input_poly=input_polynomial)
(expected_value, proof) = prover.open(evaluation_point)

verifier = FRIPCS_Verifier(polynomial_degree - 1)

if verifier.verify_proof(commitment, proof, evaluation_point, expected_value) == True:
        print(f"✓ A valid proof opening succeeded")
else:
        print(f"❌A valid opening should succeed, but it didn't")

if verifier.verify_proof(commitment, proof, evaluation_point + 1, expected_value) == False:
        print("✓ An invalid proof opening failed as expected")
else:
        print(f"❌Invalid opening of an evaluation point should not succeed, but it did!")

if verifier.verify_proof(commitment, proof, evaluation_point, expected_value - 1) == False:
        print("✓ Another invalid proof opening failed as expected")
else:
        print(f"❌Invalid opening of an evaluation point should not succeed, but it did!")

✓ A valid proof opening succeeded
✓ An invalid proof opening failed as expected
✓ Another invalid proof opening failed as expected
